In [6]:
# !pip install recordlinkage

In [7]:
import pandas as pd
from collections import defaultdict
import recordlinkage
import recordlinkage.datasets as rl_data
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score
from deepRL import DeepRL

In [8]:
# data: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/JKBULA
names = ['id', 'first', 'last', 'address', 'city']
start_index = 0
df_org_source = pd.read_csv('Q.txt', delimiter=',', names=names)
df_dup_source = pd.read_csv('A.txt', delimiter=',', encoding="ISO-8859-1", names=names)
df_dup_source = df_dup_source.iloc[[i for i in range(start_index,df_dup_source.shape[0],10)]]
assert df_org_source.shape ==  df_dup_source.shape

In [9]:
# create dict of true links
def def_value():
    return False
      
links = defaultdict(def_value)

for i in range(df_org_source.shape[0]):
  assert df_org_source.iloc[i]['id'][1:] == df_dup_source.iloc[i]['id'][1:-2]
  links[df_org_source.iloc[i]['id']] = df_dup_source.iloc[i]['id']


def true_class(org, dup):
  '''returns true if org and dup are true pair
  false otherwise'''
  if links[org] == dup:
    return True
  else:
     return False

In [122]:
# check for False in links dict
assert sum(id2 == False for id1, id2 in links.items()) == 0

In [11]:
# set index as id
df_org_source = df_org_source.set_index('id')
df_dup_source = df_dup_source.set_index('id')

In [12]:
# create separate columns for address num and street
# lower case
for df in [df_org_source, df_dup_source]:
  for col in df.columns:
    df[col] = df[col].apply(lambda word: word.lower())
  df['addr num'] = df['address'].apply(lambda x: x.split(' ')[0])
  df['addr'] = df['address'].apply(lambda x: ' '.join(x.split(' ')[1:]))
  df = df.drop('address', axis=1)

In [13]:
# create candidate pairs for source
indexer = recordlinkage.SortedNeighbourhoodIndex(
        'address', window=11
    )

candidate_pairs_source = indexer.index(df_org_source, df_dup_source)

In [14]:
# add missing true matches to source because we know all labels for source data
for id1, id2 in links.items():
  if (id1, id2) in candidate_pairs_source: # or ((id2, id1) in candidate_pairs_source):
    continue
  else:
    candidate_pairs_source = candidate_pairs_source.insert(np.random.randint(len(candidate_pairs_source)), (id1, id2))

# check for False in links dict and candidate_pairs_source
assert sum(id2 == False for id1, id2 in links.items()) == 0
assert sum(id2 == False for id1, id2 in candidate_pairs_source) == 0

In [15]:
# check that all links are in candidate_pairs_source
assert np.mean([(id1, id2) in candidate_pairs_source for id1, id2 in links.items()]) == 1
# check balance of data
print('class balance: ', np.mean([links[id1] == id2 for id1, id2 in candidate_pairs_source]))

class balance:  0.1549714852467146


In [16]:
# create list of true labels
y_source_true = [true_class(id1, id2) for id1, id2 in candidate_pairs_source]

In [17]:
# target data
data = rl_data.load_febrl4(return_links=True)
df_org_target, df_dup_target, df_links = data[0], data[1], data[2]

In [18]:
# candidate pairs for transfer learning
indexer = recordlinkage.Index()
indexer.block(left_on='postcode', right_on='postcode')
candidate_pairs_target = indexer.index(df_org_target, df_dup_target)

# create dict of true links
def def_value():
    return "Not Present"
      
links_target = defaultdict(def_value)

for org, dup in df_links:
  links_target[org] = dup

def true_class_target(org, dup):
  '''returns true if org and dup are true pair
  false otherwise'''
  if links_target[org] == dup:
    return 1
  else:
     return 0
     
# create true match list
y_target_true = [true_class_target(org, dup) for org, dup in candidate_pairs_target]

In [19]:
# check for false in links_target and candidate_pairs_target
assert sum(id2 == False for id1, id2 in links_target.items()) == 0
assert sum(id2 == False for id1, id2 in candidate_pairs_target) == 0

# Initialize DeepRL class and train embeddings

In [32]:
DL = DeepRL(df_org_source, df_dup_source, y_source_true, candidate_pairs_source, df_org_target, df_dup_target, 
            candidate_pairs_target, 50)

training embeddings complete
processing source data


100%|██████████| 64528/64528 [01:21<00:00, 790.56it/s]


processing source data complete
processing target data


100%|██████████| 28609/28609 [00:33<00:00, 847.18it/s]


processing target data complete


# Build source model and train

The source model is trained on the source data using full labels where the match/non-match status is known. We can transfer these parameters over to the target model and fine-tune.

In [33]:
DL.build_source_model(universal=False, summary=True) # bool to select type of BiGRU distance measure / bool to display summary

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
org_input (InputLayer)          [(None, 10, 10, 50)] 0                                            
__________________________________________________________________________________________________
dup_input (InputLayer)          [(None, 10, 10, 50)] 0                                            
__________________________________________________________________________________________________
tf.compat.v1.transpose_6 (TFOpL (10, None, 10, 50)   0           org_input[0][0]                  
__________________________________________________________________________________________________
tf.compat.v1.transpose_7 (TFOpL (10, None, 10, 50)   0           dup_input[0][0]                  
____________________________________________________________________________________________

None

the model is trained using the following hyperparameters

   lr = 0.001, epochs = 10, batch_size = 32



In [34]:
DL.train_source_model(0.001, 10, 32) # parameters: lr, epochs, batch

Epoch 1/10
1614/1614 [==============================] - 173s 69ms/step - loss: 0.1979 - accuracy: 0.9166
 — val_f1: 0.8575525812619503 — val_precision: 0.8229357798165138 — val_recall 0.8952095808383234
Epoch 2/10
1614/1614 [==============================] - 111s 69ms/step - loss: 0.1071 - accuracy: 0.9568
 — val_f1: 0.8164878599661208 — val_precision: 0.9401820546163849 — val_recall 0.7215568862275449
Epoch 3/10
1614/1614 [==============================] - 111s 69ms/step - loss: 0.0920 - accuracy: 0.9639
 — val_f1: 0.8904440154440154 — val_precision: 0.8621495327102804 — val_recall 0.9206586826347305
Epoch 4/10
1614/1614 [==============================] - 111s 69ms/step - loss: 0.0810 - accuracy: 0.9685
 — val_f1: 0.901324006994754 — val_precision: 0.9024512256128064 — val_recall 0.9001996007984032
Epoch 5/10
1614/1614 [==============================] - 111s 69ms/step - loss: 0.0724 - accuracy: 0.9725
 — val_f1: 0.9106614017769002 — val_precision: 0.90087890625 — val_recall 0.92065868

# Build adaptation model and train

The parameters from the source model are transfered over to the adaptation model. For training we combine data from the source and target datasets and train on two target variables. One target is match/non-match and the other is source/target datasets. Because the match/non-match status of the target data is not known we disregard the match/non-match loss for these pairs. We add a gradient reversal layer between the dataset classifier and the rest of the model. This layer makes so the BiGRU is training to trick the dataset classifier. Even though we don't have labels for the target dataset, if the outputs of the BiGRU are indistinguishable between the source and target and the match/non-match classifier is performing well on source data we have reason to believe that the performance on the target dataset will be reasonable. 

In [35]:
DL.build_adaptation_model(True) # display summary

It is important to tune the match_weight and dataset_weight so the dataset classifier isn't easily distinguishing between source and taget

In [50]:
DL.train_adaptation_model(0.001, 10, 32, 1.0, 0.1) # lr, epochs, batch_size, match_weight, dataset_weight
# trained with: lr = 0.001, epochs = 10, batch_size = 32, match_weight = 1.0, dataset_weight = 0.1

class balance on y_dataset:  0.5186749217672196
if dataset classifier in not "winning" the dataset accuracy should be close to class balance
Epoch 1/10
1486/1486 [==============================] - 164s 71ms/step - loss: 0.0829 - match_classifier_2_loss: 0.0215 - dataset_classifier_2_loss: 0.6134 - match_classifier_2_accuracy: 0.5111 - dataset_classifier_2_accuracy: 0.6489
 — val_f1: 0.928529238038984 — val_precision: 0.9538834951456311 — val_recall 0.904487917146145
Epoch 2/10
1486/1486 [==============================] - 104s 70ms/step - loss: 0.0816 - match_classifier_2_loss: 0.0199 - dataset_classifier_2_loss: 0.6165 - match_classifier_2_accuracy: 0.5116 - dataset_classifier_2_accuracy: 0.6446
 — val_f1: 0.9293044469783353 — val_precision: 0.9209039548022598 — val_recall 0.9378596087456847
Epoch 3/10
1486/1486 [==============================] - 107s 72ms/step - loss: 0.0817 - match_classifier_2_loss: 0.0201 - dataset_classifier_2_loss: 0.6161 - match_classifier_2_accuracy: 0.5114 - d

# Build target model

The dataset adaptation model is trained to make predictions on two target variables. The build_target_model() method takes the parameters from the BiGRU and the match/non-match classifier from the adaptation model and creates a new model that just predicts the match/non-match status. 

In [51]:
DL.build_target_model(transfer=True, universal=False, summary=False) 
# parameters: transfer learning vs build model from scratch/ bool for different kinds of BiGRUs / display summary

# Active / Self Learning


The active_self_learning() method pulls out the n_certain_false highest confidence non-match and n_certain_true highest confidence true match pairs, automatically labels them and adds them to the labeled target data. The method also pulls out the n_uncertain lowest confidence pairs and we label them by hand using the clerical review function. This method uses the target model to predict on unlabeled data so the build_target_model() method must be run before the active_self_learning() method can work properly

In [123]:
DL.active_self_learning(1000, 100, 10) # n_certain_false, n_certain_true, n_uncertain

# Clerical Review

The clerical_review() method allows the user to label the low confidence pairs by hand and add them to the set of labeled target data.

In [117]:
DL.clerical_review()

,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-3013-org,george,mccarthy,35,clubbe crescent,yakilli park,mentone,2540,vic,19350704,9460223
rec-3013-dup-0,georg e,mccarethy,35,clubbe crescent,yakill i park,mentone,2540,vic,19350704,9460223


Enter label (True/False) :True


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-3369-org,callum,lowry,52,hallen close,ruglen,picnic point,2000,vic,19971123,3987913
rec-3369-dup-0,callum,lowry,52,ruglen,hallen close,picnic point,2000,vic,19971123,3987913


Enter label (True/False) :True


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-4496-org,hari,warnock,8,jansz crescent,brentwood vlge,broadmeadows,2486,qld,19350219,7539077
rec-4496-dup-0,warnock,hark,NaN,jansz cr escent,brentwood vlge,broadmeadows,2486,NaN,19350219,7539077


...

Enter label (True/False) :False


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-4136-org,NaN,vincent,47,kent street,blue hills,forest hill,4655,vic,19190304,5807175
rec-4136-dup-0,NaN,vincent,47,kentstreet,NaN,forest hlil,4655,vic,19190304,5807175


Enter label (True/False) :True


,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,date_of_birth,soc_sec_id
rec-4552-org,sophie,mason,22,sturgess place,acadia,pacific paradise,3215,nsw,19790301,2531438
rec-3801-dup-0,sophie,hage,18,riley clwose,rocklea,seaforvh,3215,nsw,19771204,9725410


Enter label (True/False) :False
all data labeled


# Training target model

The train_target_model() method can be used to train the target model just using the labeled target data that was either submitting when the DeepRL class was initialized, acquired through self/active learning or a combination of the two. This method should only be used after a sufficient amount of labeled target data is available. If there is minimal training data and this method is run, it can cause the model to easily overfit and perform poorly on the unlabeled data. 

In [119]:
DL.train_target_model(0.0001, 1, 32) # lr, epochs, batch_size, optional: y_labels, indices for labels in candidate pairs

314/314 [==============================] - 76s 68ms/step - loss: 0.0042 - accuracy: 0.9981
 — val_f1: 0.9928057553956835 — val_precision: 0.9857142857142858 — val_recall 1.0


# Notes on overall training process

The DeepRL class can be used in many different ways. If there is sufficient labeled target data the user can just initialize the class and immediately train on the target data. In situations where there is a limited amount of labeled target data, dataset adaptation/self/active learning should be used. In this case it might be wise to initially iterate between the train_dataset_adaptation method and self_active_learning methods. As labeled target data is accumulated this data is added to the training data for the dataset adaptation model, so during each additional round of self/active learning more labeled target data will be available during training with the train_dataset_adaptation() method. We need to be careful not to switch to the target model for self/active learning too early in the process. If the target model is trained when there is a small amount of labeled target data, the model can perform very poorly. If this model has poor performance and it is used for self learning, the data that is automatically labeled can be labeled incorrectly. This will lead to training on incorrectly labeled data which will be very detrimental to model performance. In preliminary experimentation it seems around 4000 labeled target pairs leads to reasonable performance when training the target model. After there is sufficient labeled target data the user can switch over to iterating between train_target_model() and self_active_learning(). 

In [118]:
# in this contrived scenario where we actually do know the true match status of the target data we can check 
# if there are any pairs that have been added to labeled target data that were labeled incorrectly
# we can see below we have accuracy, precision and recall all equal to 1
# in a real world scenario this step would not be possible
y_true_f = [true_class_target(candidate_pairs_target[i][0], candidate_pairs_target[i][1]) for i in DL.y_target_indices]
y_pred_f = DL.y_target

print('accuracy', accuracy_score(y_true_f, y_pred_f))
print('recall', recall_score(y_true_f, y_pred_f))
print('precision', precision_score(y_true_f, y_pred_f))

accuracy 1.0
recall 1.0
precision 1.0


In [107]:
# check accuracy, recall and precision for non-labeled target data
y_target_pred = DL.model_target.predict(x=[np.delete(DL.org_target_embed, DL.y_target_indices, axis=0),
                                           np.delete(DL.dup_target_embed, DL.y_target_indices, axis=0)])
y_target_pred = y_target_pred > 0.5
y_target_true_delete = np.delete(y_target_true, DL.y_target_indices, axis=0)
print('accuracy', accuracy_score(y_target_true_delete, y_target_pred))
print('recall', recall_score(y_target_true_delete, y_target_pred))
print('precision', precision_score(y_target_true_delete, y_target_pred))

accuracy 0.9767716324231532
recall 0.9275715155203895
precision 0.9398704902867715


In [121]:
# the results shown here are after 4 training self/active learning iterations, 
# the last of these iterations was just hand labeling 50 pairs